In [10]:
import tensorflow
def test(hp):
    return tensorflow.keras.models.Sequential()

In [11]:
import keras_tuner as kt

tuner = kt.RandomSearch(test)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


2022-07-08 09:26:29.273876: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-08 09:26:29.335876: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-08 09:26:29.336012: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-08 09:26:29.336290: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [12]:
tuner.search(callbacks=[])


Search: Running Trial #1

default configuration



RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [1]:
# Basics
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# TensorFlow/Keras
import keras_tuner as kt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, RMSprop

# Custom
import sys
sys.path.insert(0, '/home/lcastellazzi/MDM32/src/utils')
from preprocessing import TraceHandler
from nicv import nicv
import constants
from postprocessing import Evaluator


# Suppress TensorFlow messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # 1 for INFO, 2 for INFO & WARNINGs, 3 for INFO & WARNINGs & ERRORs

In [2]:
paths = {'train': '/prj/side_channel/PinataTraces/CURR/D1-K1_50k_500MHz + Resampled at 168MHz.trs', # D1_K1
         'test' : '/prj/side_channel/PinataTraces/CURR/D1-K2_50k_500MHz + Resampled at 168MHz.trs'} # D1_K2

trace_handlers = {key: TraceHandler(path) for key, path in paths.items()}

Labeling traces: 100%|██████████| 50000/50000 [00:21<00:00, 2306.87it/s]


In [3]:
BYTE_IDX = 0
VAL_PERC = 0.1
N_CLASSES = 256

x_train_tot = trace_handlers['train'].get_traces()
y_train_tot = trace_handlers['train'].get_specific_labels(BYTE_IDX)
y_train_tot_cat = y_train_cat = to_categorical(y_train_tot, N_CLASSES)

x_train, x_val, y_train, y_val = trace_handlers['train'].generate_train_val(BYTE_IDX, val_perc=VAL_PERC) 
y_train_cat = to_categorical(y_train, N_CLASSES)
y_val_cat = to_categorical(y_val, N_CLASSES)

x_test, y_test = trace_handlers['test'].generate_test(BYTE_IDX) 
y_test_cat = to_categorical(y_test, N_CLASSES)

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop

INPUT_SIZE = len(x_train[0])
N_CLASSES = 256
BATCH_SIZE = 256
LOSS = 'categorical_crossentropy'

def build_model(hp):
    
    # BATCH_NORM = hp.Boolean('batch_norm')
    
    NUM_LAYERS = hp.Int('num_layers',
                        min_value=1,
                        max_value=10,
                        sampling='log')
    
    NUM_NEURONS = hp.Choice('num_neurons', [100, 200, 300, 400])
    
#     DROPOUT = hp.Boolean('dropout')
    
#     if DROPOUT:
#         DROPOUT_RATE = hp.Float('dropout_rate', 
#                                 min_value=0.1,
#                                 max_value=0.9,
#                                 # step=0.1,
#                                 sampling='log')    
        
    LR = hp.Float('lr', 
                  min_value=1e-6,
                  max_value=1e-3,
                  sampling='log')
    
    ADAM_OPT = hp.Boolean('adam_opt')
    
    if ADAM_OPT:
        OPTIMIZER = Adam(learning_rate=LR)
    else:
        OPTIMIZER = RMSprop(learning_rate=LR)
    
    ############################################################
    
    # Init model
    model = Sequential()
    
    # Input
    model.add(Dense(INPUT_SIZE, activation='relu'))
    
    # Batch Normalization
    model.add(BatchNormalization())
    
    # Hidden
    for _ in range(NUM_LAYERS):
        model.add(Dense(NUM_NEURONS, activation='relu'))
    
    # # Dropout
    # if DROPOUT:
    #     model.add(Dropout(DROPOUT_RATE))
    
    # Output
    model.add(Dense(N_CLASSES, activation='softmax'))
    
    model.compile(optimizer=OPTIMIZER,
                  loss=LOSS,
                  metrics=['accuracy'])
    
    return model

In [17]:
tuner = kt.RandomSearch(hypermodel=build_model,                                               # Function that generates the model 
                        objective="val_loss",                                                 # What to optimized   
                        max_trials=20,                                                        # Number of different hyperparameters configurations to try
                        executions_per_trial=2,                                               # Number of different models to build and fit (same hps) in each trial (robustness purposes)
                        overwrite=True,                                                       # Wheter or not overwrite the results of the previous searc
                        directory='/home/lcastellazzi/DL-SCA/notebooks/keras_tuner_results',  # Main results folder
                        project_name='first_test')                                            # Specific subfolder of the main results folder

In [18]:
tuner.search(x_train, 
             y_train_cat, 
             epochs=50, 
             validation_data=(x_val, y_val_cat),
             batch_size=BATCH_SIZE)

best_model = tuner.get_best_models()[0]

Trial 2 Complete [00h 05m 06s]
val_loss: 5.03868293762207

Best val_loss So Far: 4.2068541049957275
Total elapsed time: 00h 06m 46s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
10                |4                 |num_layers
300               |300               |num_neurons
0.00067629        |0.00092939        |lr
False             |True              |adam_opt

Epoch 1/50
176/176 [==============================] - 4s 16ms/step - loss: 5.2035 - accuracy: 0.0109 - val_loss: 11.4508 - val_accuracy: 0.0036
Epoch 2/50
176/176 [==============================] - 3s 15ms/step - loss: 4.8791 - accuracy: 0.0169 - val_loss: 5.0038 - val_accuracy: 0.0116
Epoch 3/50
176/176 [==============================] - 1s 4ms/step - loss: 4.7123 - accuracy: 0.0238 - val_loss: 5.3617 - val_accuracy: 0.0146
Epoch 4/50
176/176 [==============================] - 1s 4ms/step - loss: 4.4102 - accuracy: 0.0409 - val_loss: 5.0746 - val_accuracy: 0.0176
Epoch 5/50
176/176 [=========

KeyboardInterrupt: 

In [ ]:
NUM_EPOCHS = 100

history = best_model.fit(x_train_tot,
                         y_train_tot_cat,
                         epochs=NUM_EPOCHS,
                         batch_size=BATCH_SIZE,
                         # callbacks=[es],
                         verbose=1)

In [ ]:
plt.plot(history.history['loss'], label='train_loss')
# plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='train_acc')
# plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

In [ ]:
label_probs = best_model.predict(x_test)

In [ ]:
plaintexts = trace_handlers['test'].get_plaintexts()
true_key_byte = constants.KEYS['K2'][BYTE_IDX]

evaluator = Evaluator(label_probs, plaintexts, BYTE_IDX)
evaluator.rank_key_bytes()
true_key_byte_rank = evaluator.get_true_key_byte_rank(true_key_byte)

true_key_byte_rank